## Jeopardy

In [1]:
# dependencies
import random
import pandas as pd
import requests
import json
import numpy as np
import glob
import json
import csv
import os

In [30]:
# read in jeopardy.csv1
file_jeo1 = "jeopardy_1.csv"
data_jeo1 = pd.read_csv(file_jeo1,index_col=None, header=0)
data_jeo1.head(2)

,gid,airdate,rnd,category,text,answer,value
0,346,1983-09-18,1,WORLD GEOGRAPHY,"This country's counties include Kerry, Kilken...",Ireland,25
1,346,1983-09-18,1,BEST-SELLERS,This Jacqueline Susann novel is the all-time ...,Valley of the Dolls,25


In [31]:
data_jeo1 = data_jeo1.rename(columns={"text":"Question", "answer": "Answer", "value": "Value", "category":"Category", 
                                      "airdate":"Air_Date"})
data_jeo1.columns
data_jeo1.head(2)

,gid,Air_Date,rnd,Category,Question,Answer,Value
0,346,1983-09-18,1,WORLD GEOGRAPHY,"This country's counties include Kerry, Kilken...",Ireland,25
1,346,1983-09-18,1,BEST-SELLERS,This Jacqueline Susann novel is the all-time ...,Valley of the Dolls,25


In [32]:
data_jeo1 = data_jeo1[['gid', 'Air_Date', 'rnd', 'Category', 'Value', 'Question', 'Answer']]
data_jeo1.head(2)

,gid,Air_Date,rnd,Category,Value,Question,Answer
0,346,1983-09-18,1,WORLD GEOGRAPHY,25,"This country's counties include Kerry, Kilken...",Ireland
1,346,1983-09-18,1,BEST-SELLERS,25,This Jacqueline Susann novel is the all-time ...,Valley of the Dolls


In [5]:
# read in jeopardy.csv2 cp1252
data_jeo2 = pd.read_csv("jeopardy_2.csv" , encoding="ISO-8859-1")
data_jeo2.columns

Index(['Show_Number', 'Air_Date', 'Round', 'Category', ' Value ', 'Question',
       'Answer'],
      dtype='object')

In [126]:
# merge tables
complete_jeopardy = pd.merge(data_jeo1, data_jeo2, on="Question", how="left")
complete_jeopardy.head(2)

,gid,Air_Date_x,rnd,Category_x,Value,Question,Answer_x,Show_Number,Air_Date_y,Round,Category_y,Value,Answer_y
0,346,1983-09-18,1,WORLD GEOGRAPHY,25,"This country's counties include Kerry, Kilken...",Ireland,NaN,NaN,NaN,NaN,NaN,NaN
1,346,1983-09-18,1,BEST-SELLERS,25,This Jacqueline Susann novel is the all-time ...,Valley of the Dolls,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
complete_jeopardy.columns

Index(['gid', 'Air_Date_x', 'rnd', 'Category_x', 'Value', 'Question',
       'Answer_x', 'Show_Number', 'Air_Date_y', 'Round', 'Category_y',
       ' Value ', 'Answer_y'],
      dtype='object')

In [127]:
complete_jeopardy = complete_jeopardy.rename(columns={'gid': 'Game ID', 'Air_Date_x': "Air Date", 'rnd': "Rnd", 
                                                     'Category_x': "Category", 'Value': 'Value', 'Question': 'Question', 
                                                      'Answer_x':'Answer', 'Show_Number': 'Show Number', 
                                                      'Round':'Jeopardy_Double'})
complete_jeopardy.head(2)

,Game ID,Air Date,Rnd,Category,Value,Question,Answer,Show Number,Air_Date_y,Jeopardy_Double,Category_y,Value,Answer_y
0,346,1983-09-18,1,WORLD GEOGRAPHY,25,"This country's counties include Kerry, Kilken...",Ireland,NaN,NaN,NaN,NaN,NaN,NaN
1,346,1983-09-18,1,BEST-SELLERS,25,This Jacqueline Susann novel is the all-time ...,Valley of the Dolls,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
complete_jeopardy.columns

Index(['Game ID', 'Air Date', 'Rnd', 'Category', 'Value', 'Question', 'Answer',
       'Show Number', 'Air_Date_y', 'Jeopardy_Double', 'Category_y', ' Value ',
       'Answer_y'],
      dtype='object')

In [128]:
complete_jeopardy = complete_jeopardy[['Game ID', 'Show Number', 'Jeopardy_Double','Rnd', 'Air Date','Category', 'Value', 
                                       'Question', 'Answer']]
complete_jeopardy.head(2)

,Game ID,Show Number,Jeopardy_Double,Rnd,Air Date,Category,Value,Question,Answer
0,346,NaN,NaN,1,1983-09-18,WORLD GEOGRAPHY,25,"This country's counties include Kerry, Kilken...",Ireland
1,346,NaN,NaN,1,1983-09-18,BEST-SELLERS,25,This Jacqueline Susann novel is the all-time ...,Valley of the Dolls


In [129]:
complete_jeopardy.drop(['Show Number'], axis=1)
complete_jeopardy.head(2)

,Game ID,Show Number,Jeopardy_Double,Rnd,Air Date,Category,Value,Question,Answer
0,346,NaN,NaN,1,1983-09-18,WORLD GEOGRAPHY,25,"This country's counties include Kerry, Kilken...",Ireland
1,346,NaN,NaN,1,1983-09-18,BEST-SELLERS,25,This Jacqueline Susann novel is the all-time ...,Valley of the Dolls


In [130]:
complete_jeopardy['Year'] = complete_jeopardy['Air Date'].str[:5]
complete_jeopardy.head(2)

,Game ID,Show Number,Jeopardy_Double,Rnd,Air Date,Category,Value,Question,Answer,Year
0,346,NaN,NaN,1,1983-09-18,WORLD GEOGRAPHY,25,"This country's counties include Kerry, Kilken...",Ireland,1983
1,346,NaN,NaN,1,1983-09-18,BEST-SELLERS,25,This Jacqueline Susann novel is the all-time ...,Valley of the Dolls,1983


In [131]:
complete_jeopardy['Answer'].value_counts().head(10)

 Australia     335
 China         330
 Chicago       316
 France        316
 Japan         310
 India         288
 Spain         277
 California    274
 Canada        273
 Alaska        256
Name: Answer, dtype: int64

In [25]:
complete_jeopardy.to_csv("complete_jeopardy.csv", index=False)

## Facial Hair

In [197]:
hair_file = pd.read_csv("university-graduate-facial-hair-styles_2.csv" , encoding="ISO-8859-1")
facial_hair = pd.DataFrame(hair_file)
facial_hair.set_index('Year', inplace=True)
facial_hair.head()

,Cleanshaven,Full Beard/Chinstrap,Goatee,Moustache,Moustache (with Muttonchops),Moustache (with Sideburns),Muttonchops,Sideburns
Year,,,,,,,,
2008,127,20,11,1,0,0,1,43
2007,227,25,21,0,0,0,10,89
2006,236,18,26,1,0,0,6,66
2005,325,17,34,1,0,0,5,110
2004,353,17,61,1,0,0,6,91


## Divorce and Marriage

In [162]:
rates_file = pd.read_csv("national_marriage_divorce_rates_00-16.csv" , encoding="ISO-8859-1")
dm_rates = pd.DataFrame(rates_file)
dm_rates.set_index('Year', inplace=True)
dm_rates.head()

,Marriages,Population,"Rate per 1,000 total population",Divorces & annulments,Population.1,"Rate per 1,000 total population.1"
Year,,,,,,
2016,"2,245,404","323,127,513",6.9,"827,261","257,904,548",3.2
2015,"2,221,579","321,418,820",6.9,"800,909","258,518,265",3.1
2014,"2,140,272","308,759,713",6.9,"813,862","256,483,624",3.2
2013,"2,081,301","306,136,672",6.8,"832,157","254,408,815",3.3
2012,"2,131,000","313,914,040",6.8,"851,000","248,041,986",3.4


## Ted Talks

In [189]:
ted_file = pd.read_csv("TED_Talks_by_ID_plus-transcripts-and-LIWC-and-MFT-plus-views.csv" , encoding="ISO-8859-1")
ted = pd.DataFrame(ted_file)
ted.head()

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,...,harm_vice,fairness_virtue,fairness_vice,ingroup_virtue,ingroup_vice,authority_virtue,authority_vice,purity_virtue,purity_vice,morality_general
0,1,Al Gore,Averting the climate crisis,http://www.ted.com/talks/view/id/1,With the same humor and humanity he exuded in ...,http://www.ted.com/talks/view/id/1/transcript?...,2,2006,TED2006,0:16:17,...,0.04,0.00,0.00,0.48,0.00,0.22,0.0,0.00,0.00,0.22
1,2,Amy Smith,Simple designs to save a life,http://www.ted.com/talks/view/id/2,Fumes from indoor cooking fires kill more than...,http://www.ted.com/talks/view/id/2/transcript?...,2,2006,TED2006,0:15:06,...,0.04,0.00,0.00,0.30,0.00,0.11,0.0,0.11,0.04,0.15
2,3,Ashraf Ghani,How to rebuild a broken state,http://www.ted.com/talks/view/id/3,Ashraf Ghani's passionate and powerful 10-minu...,http://www.ted.com/talks/view/id/3/transcript?...,7,2005,TEDGlobal 2005,0:18:45,...,0.12,0.16,0.04,0.32,0.12,0.20,0.0,0.04,0.04,0.08
3,4,Burt Rutan,The real future of space exploration,http://www.ted.com/talks/view/id/4,"In this passionate talk, legendary spacecraft ...",http://www.ted.com/talks/view/id/4/transcript?...,2,2006,TED2006,0:19:37,...,0.19,0.00,0.00,0.19,0.00,0.10,0.0,0.00,0.00,0.16
4,5,Chris Bangle,Great cars are great art,http://www.ted.com/talks/view/id/5,American designer Chris Bangle explains his ph...,http://www.ted.com/talks/view/id/5/transcript?...,2,2002,TED2002,0:20:04,...,0.05,0.03,0.00,0.39,0.00,0.05,0.0,0.00,0.03,0.13


In [190]:
ted.columns

Index(['id', 'speaker', 'headline', 'URL', 'description', 'transcript_URL',
       'month_filmed', 'year_filmed', 'event', 'duration',
       ...
       'harm_vice', 'fairness_virtue', 'fairness_vice', 'ingroup_virtue',
       'ingroup_vice', 'authority_virtue', 'authority_vice', 'purity_virtue',
       'purity_vice', 'morality_general'],
      dtype='object', length=123)

In [192]:
ted_yr = ted.rename(columns={'year_filmed': 'Year'})
ted_yr.head()

,id,speaker,headline,URL,description,transcript_URL,month_filmed,Year,event,duration,...,harm_vice,fairness_virtue,fairness_vice,ingroup_virtue,ingroup_vice,authority_virtue,authority_vice,purity_virtue,purity_vice,morality_general
0,1,Al Gore,Averting the climate crisis,http://www.ted.com/talks/view/id/1,With the same humor and humanity he exuded in ...,http://www.ted.com/talks/view/id/1/transcript?...,2,2006,TED2006,0:16:17,...,0.04,0.00,0.00,0.48,0.00,0.22,0.0,0.00,0.00,0.22
1,2,Amy Smith,Simple designs to save a life,http://www.ted.com/talks/view/id/2,Fumes from indoor cooking fires kill more than...,http://www.ted.com/talks/view/id/2/transcript?...,2,2006,TED2006,0:15:06,...,0.04,0.00,0.00,0.30,0.00,0.11,0.0,0.11,0.04,0.15
2,3,Ashraf Ghani,How to rebuild a broken state,http://www.ted.com/talks/view/id/3,Ashraf Ghani's passionate and powerful 10-minu...,http://www.ted.com/talks/view/id/3/transcript?...,7,2005,TEDGlobal 2005,0:18:45,...,0.12,0.16,0.04,0.32,0.12,0.20,0.0,0.04,0.04,0.08
3,4,Burt Rutan,The real future of space exploration,http://www.ted.com/talks/view/id/4,"In this passionate talk, legendary spacecraft ...",http://www.ted.com/talks/view/id/4/transcript?...,2,2006,TED2006,0:19:37,...,0.19,0.00,0.00,0.19,0.00,0.10,0.0,0.00,0.00,0.16
4,5,Chris Bangle,Great cars are great art,http://www.ted.com/talks/view/id/5,American designer Chris Bangle explains his ph...,http://www.ted.com/talks/view/id/5/transcript?...,2,2002,TED2002,0:20:04,...,0.05,0.03,0.00,0.39,0.00,0.05,0.0,0.00,0.03,0.13


# World Population

## World Population website scrape

In [34]:
url = 'http://www.worldometers.info/world-population/world-population-projections/'

In [35]:
worldpop_future = pd.read_html(url)
type(worldpop_future)

list

In [124]:
pop_df = worldpop_future[0]
# pop_df
pop_df = pop_df[['Year', 'World Population']]
pop_df.head()

,Year,World Population
0,2018,7632819325
1,2019,7714576923
2,2020,7795482309
3,2021,7875464634
4,2022,7954468982


In [121]:
# read in world population excel
worldpop = pd.read_excel("WPP2017.xlsx", index_col=None, header=0)
worldpop.to_csv('world_pop.csv', index=False)
worldpop = pd.read_csv('world_pop.csv', index_col=None, header=0)
worldpop = pd.DataFrame(worldpop)
worldpop.drop([0, 1, 2, 3, 4, 5 ,6 ,7 , 8, 9 , 10, 11, 12, 13], inplace=True)
pop_trans = worldpop.transpose()
pop_trans.drop(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 2'], inplace=True)
# pop_trans
population = pop_trans[[14,15]]
# population
population = population.rename(columns={14 : 'Year', 15 : 'World Population'})
population
# del population['index']
population.set_index('Year')
population.columns

Index(['Year', 'World Population'], dtype='object')

In [204]:
ttl_pop = pd.concat([population , pop_df], ignore_index=True)
ttl_pop.set_index('Year', inplace=True)
ttl_pop.head()

,World Population
Year,
1950,2536274.721
1951.0,2.58382e+06
1952.0,2.63058e+06
1953.0,2.67723e+06
1954.0,2.7243e+06


In [205]:
ttl_pop.to_csv("complete_worldpop.csv", index=False)

## load all datasets into mySQL

In [215]:
 # Dependencies
from sqlalchemy import create_engine

In [213]:
rds_connection_string = "<inser user name>:<insert password>@127.0.0.1/customer_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [214]:
engine.table_names() 